[View in Colaboratory](https://colab.research.google.com/github/samiul272/AI-Stuff/blob/master/OpenBanglaCapsNet.ipynb)

*This code is adapted for use in Google Colaboratory from Kevin Mader's [Capsulenet implementation](https://www.kaggle.com/kmader/capsulenet-on-mnist) which in turn is adapted from Xifeng Guo's github [code](https://github.com/XifengGuo/CapsNet-Keras/blob/master/capsulenet.py). These codes are based on Sara Sabour's paper [Dynamic Routing Between Capsules](https://papers.nips.cc/paper/6975-dynamic-routing-between-capsules.pdf).*

**First we install fuse drive. This will enable us to use our google drive data directly in colab without needing to get the Id  every time**

In [2]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 16712 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .../0


Creating config file /etc/apt/apt.conf.d/20auto-upgrades with new version

Creating config file /etc/apt/apt.conf.d/50unattended-upgrades with new version
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up dirmngr (2.1.15-1ubuntu8) ...
Setting up cron (3.0pl1-128ubuntu5) ...
Adding group `crontab' (GID 102) ...
Done.
update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
update-rc.d: warning: stop runlevel arguments (1) do not match cron Default-Stop values (none)
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Setting up kmod (24-1ubuntu2) ...
Setting up libdbus-glib-1-2:amd64 (0.108-2) ...
Setting up python3-gi (3.24.1-2build1) ...
Setting up module-init-tools (24-1ubuntu2) ...
Setting up python3-software-properties (0.96.24.17) ...
Setting up dbus (1.10.22-1ubuntu1) ...
Setting

**Next we get our tokens. You will get a link that you'd need to follow and sign in to your google account and allow access.**

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

**We will do the same for the fuse library**

In [4]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


**Next we make directory drive. This will point to your drive home page. You can change the directory by using *ls* command and see what is inside **

In [7]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive -o nonempty

print('Files in Drive:')
!ls drive/Datasets/Final_DB

Files in Drive:
capsulelayers.py	 testing-a	testing-e	training-c
capsulelayers.pyc	 testing-a.csv	testing-e.csv	training-c.csv
capsulenet.py		 testing-b	testing-f	training-d
loaders.py		 testing-b.csv	testing-f.csv	training-d.csv
loaders.pyc		 testing-c	training-a	training-e
OpenBanglaCapsNet.ipynb  testing-c.csv	training-a.csv	training-e.csv
__pycache__		 testing-d	training-b	utils.py
stats.xlsx		 testing-d.csv	training-b.csv	utils.pyc


**Now we get to the actual code work. We start by importing all the things we need.**

In [8]:
import numpy as np
import os
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks
from keras.utils.vis_utils import plot_model

Using TensorFlow backend.


**Next the special layers for capsule net are defined**

In [0]:
import keras.backend as K
import tensorflow as tf
from keras import initializers, layers

class Length(layers.Layer):
    """
    Compute the length of vectors. This is used to compute a Tensor that has the same shape with y_true in margin_loss
    inputs: shape=[dim_1, ..., dim_{n-1}, dim_n]
    output: shape=[dim_1, ..., dim_{n-1}]
    """
    def call(self, inputs, **kwargs):
        return K.sqrt(K.sum(K.square(inputs), -1))

    def compute_output_shape(self, input_shape):
        return input_shape[:-1]

class Mask(layers.Layer):
    """
    Mask a Tensor with shape=[None, d1, d2] by the max value in axis=1.
    Output shape: [None, d2]
    """
    def call(self, inputs, **kwargs):
        # use true label to select target capsule, shape=[batch_size, num_capsule]
        if type(inputs) is list:  # true label is provided with shape = [batch_size, n_classes], i.e. one-hot code.
            assert len(inputs) == 2
            inputs, mask = inputs
        else:  # if no true label, mask by the max length of vectors of capsules
            x = inputs
            # Enlarge the range of values in x to make max(new_x)=1 and others < 0
            x = (x - K.max(x, 1, True)) / K.epsilon() + 1
            mask = K.clip(x, 0, 1)  # the max value in x clipped to 1 and other to 0

        # masked inputs, shape = [batch_size, dim_vector]
        inputs_masked = K.batch_dot(inputs, mask, [1, 1])
        return inputs_masked

    def compute_output_shape(self, input_shape):
        if type(input_shape[0]) is tuple:  # true label provided
            return tuple([None, input_shape[0][-1]])
        else:
            return tuple([None, input_shape[-1]])


def squash(vectors, axis=-1):
    """
    The non-linear activation used in Capsule. It drives the length of a large vector to near 1 and small vector to 0
    :param vectors: some vectors to be squashed, N-dim tensor
    :param axis: the axis to squash
    :return: a Tensor with same shape as input vectors
    """
    s_squared_norm = K.sum(K.square(vectors), axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / K.sqrt(s_squared_norm)
    return scale * vectors


class CapsuleLayer(layers.Layer):
    """
    The capsule layer. It is similar to Dense layer. Dense layer has `in_num` inputs, each is a scalar, the output of the 
    neuron from the former layer, and it has `out_num` output neurons. CapsuleLayer just expand the output of the neuron
    from scalar to vector. So its input shape = [None, input_num_capsule, input_dim_vector] and output shape = \
    [None, num_capsule, dim_vector]. For Dense Layer, input_dim_vector = dim_vector = 1.
    
    :param num_capsule: number of capsules in this layer
    :param dim_vector: dimension of the output vectors of the capsules in this layer
    :param num_routings: number of iterations for the routing algorithm
    """
    def __init__(self, num_capsule, dim_vector, num_routing=3,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_vector = dim_vector
        self.num_routing = num_routing
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)

    def build(self, input_shape):
        assert len(input_shape) >= 3, "The input Tensor should have shape=[None, input_num_capsule, input_dim_vector]"
        self.input_num_capsule = input_shape[1]
        self.input_dim_vector = input_shape[2]

        # Transform matrix
        self.W = self.add_weight(shape=[self.input_num_capsule, self.num_capsule, self.input_dim_vector, self.dim_vector],
                                 initializer=self.kernel_initializer,
                                 name='W')

        # Coupling coefficient. The redundant dimensions are just to facilitate subsequent matrix calculation.
        self.bias = self.add_weight(shape=[1, self.input_num_capsule, self.num_capsule, 1, 1],
                                    initializer=self.bias_initializer,
                                    name='bias',
                                    trainable=False)
        self.built = True

    def call(self, inputs, training=None):
        # inputs.shape=[None, input_num_capsule, input_dim_vector]
        # Expand dims to [None, input_num_capsule, 1, 1, input_dim_vector]
        inputs_expand = K.expand_dims(K.expand_dims(inputs, 2), 2)

        # Replicate num_capsule dimension to prepare being multiplied by W
        # Now it has shape = [None, input_num_capsule, num_capsule, 1, input_dim_vector]
        inputs_tiled = K.tile(inputs_expand, [1, 1, self.num_capsule, 1, 1])

        """  
        # Compute `inputs * W` by expanding the first dim of W. More time-consuming and need batch_size.
        # Now W has shape  = [batch_size, input_num_capsule, num_capsule, input_dim_vector, dim_vector]
        w_tiled = K.tile(K.expand_dims(self.W, 0), [self.batch_size, 1, 1, 1, 1])
        
        # Transformed vectors, inputs_hat.shape = [None, input_num_capsule, num_capsule, 1, dim_vector]
        inputs_hat = K.batch_dot(inputs_tiled, w_tiled, [4, 3])
        """
        # Compute `inputs * W` by scanning inputs_tiled on dimension 0. This is faster but requires Tensorflow.
        # inputs_hat.shape = [None, input_num_capsule, num_capsule, 1, dim_vector]
        inputs_hat = tf.scan(lambda ac, x: K.batch_dot(x, self.W, [3, 2]),
                             elems=inputs_tiled,
                             initializer=K.zeros([self.input_num_capsule, self.num_capsule, 1, self.dim_vector]))
        """
        # Routing algorithm V1. Use tf.while_loop in a dynamic way.
        def body(i, b, outputs):
            c = tf.nn.softmax(self.bias, dim=2)  # dim=2 is the num_capsule dimension
            outputs = squash(K.sum(c * inputs_hat, 1, keepdims=True))
            b = b + K.sum(inputs_hat * outputs, -1, keepdims=True)
            return [i-1, b, outputs]

        cond = lambda i, b, inputs_hat: i > 0
        loop_vars = [K.constant(self.num_routing), self.bias, K.sum(inputs_hat, 1, keepdims=True)]
        _, _, outputs = tf.while_loop(cond, body, loop_vars)
        """
        # Routing algorithm V2. Use iteration. V2 and V1 both work without much difference on performance
        assert self.num_routing > 0, 'The num_routing should be > 0.'
        for i in range(self.num_routing):
            c = tf.nn.softmax(self.bias, dim=2)  # dim=2 is the num_capsule dimension
            # outputs.shape=[None, 1, num_capsule, 1, dim_vector]
            outputs = squash(K.sum(c * inputs_hat, 1, keepdims=True))

            # last iteration needs not compute bias which will not be passed to the graph any more anyway.
            if i != self.num_routing - 1:
                # self.bias = K.update_add(self.bias, K.sum(inputs_hat * outputs, [0, -1], keepdims=True))
                self.bias += K.sum(inputs_hat * outputs, -1, keepdims=True)
            # tf.summary.histogram('BigBee', self.bias)  # for debugging
        return K.reshape(outputs, [-1, self.num_capsule, self.dim_vector])

    def compute_output_shape(self, input_shape):
        return tuple([None, self.num_capsule, self.dim_vector])


def PrimaryCap(inputs, dim_vector, n_channels, kernel_size, strides, padding):
    """
    Apply Conv2D `n_channels` times and concatenate all capsules
    :param inputs: 4D tensor, shape=[None, width, height, channels]
    :param dim_vector: the dim of the output vector of capsule
    :param n_channels: the number of types of capsules
    :return: output tensor, shape=[None, num_capsule, dim_vector]
    """
    output = layers.Conv2D(filters=dim_vector*n_channels, kernel_size=kernel_size, strides=strides, padding=padding)(inputs)
    outputs = layers.Reshape(target_shape=[-1, dim_vector])(output)
    return layers.Lambda(squash)(outputs)

**CapsNet is the actual model. We define it here. Unlike training a regular machine learning model, the input-output relation in capsnet is not defined by the standard $X$â$$ relation but rather an $(X,y)$ â $(y,X)$model meaning it attempts to predict the class from the image, and then at the same time, using the same capsule reconstruct the image from the class. This follows the conditional generative adverserial network model and the task of reconstructing the model helps the model understand the image better.**y 

In [0]:
from keras import layers, models
from keras import backend as K
from keras.utils import to_categorical
def CapsNet(input_shape, n_class, num_routing):
    """
    A Capsule Network on MNIST.
    :param input_shape: data shape, 4d, [None, width, height, channels]
    :param n_class: number of classes
    :param num_routing: number of routing iterations
    :return: A Keras Model with 2 inputs and 2 outputs
    """
    x = layers.Input(shape=input_shape)

    # Layer 1: Just a conventional Conv2D layer
    conv1 = layers.Conv2D(filters=256, kernel_size=9, strides=1, padding='valid', activation='relu', name='conv1')(x)

    # Layer 2: Conv2D layer with `squash` activation, then reshape to [None, num_capsule, dim_vector]
    primarycaps = PrimaryCap(conv1, dim_vector=8, n_channels=32, kernel_size=9, strides=2, padding='valid')

    # Layer 3: Capsule layer. Routing algorithm works here.
    digitcaps = CapsuleLayer(num_capsule=n_class, dim_vector=16, num_routing=num_routing, name='digitcaps')(primarycaps)

    # Layer 4: This is an auxiliary layer to replace each capsule with its length. Just to match the true label's shape.
    # If using tensorflow, this will not be necessary. :)
    out_caps = Length(name='out_caps')(digitcaps)

    # Decoder network.
    y = layers.Input(shape=(n_class,))
    masked = Mask()([digitcaps, y])  # The true label is used to mask the output of capsule layer.
    x_recon = layers.Dense(512, activation='relu')(masked)
    x_recon = layers.Dense(1024, activation='relu')(x_recon)
    x_recon = layers.Dense(784, activation='sigmoid')(x_recon)
    x_recon = layers.Reshape(target_shape=[28, 28, 1], name='out_recon')(x_recon)

    # two-input-two-output keras Model
    return models.Model([x, y], [out_caps, x_recon])

**This defines the loss function**

In [0]:
def margin_loss(y_true, y_pred):
    """
    Margin loss for Eq.(4). When y_true[i, :] contains not just one `1`, this loss should work too. Not test it.
    :param y_true: [None, n_classes]
    :param y_pred: [None, num_capsule]
    :return: a scalar loss value.
    """
    L = y_true * K.square(K.maximum(0., 0.9 - y_pred)) + \
        0.5 * (1 - y_true) * K.square(K.maximum(0., y_pred - 0.1))

    return K.mean(K.sum(L, 1))

**We take a look at the summary of the model. Around 8 million parameters to optimize. This  will increase exponentially if you use lager images **

In [13]:
model = CapsNet(input_shape=[28, 28, 1],
                n_class=10,
                num_routing=3)
model.summary()

Instructions for updating:
dim is deprecated, use axis instead
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 20, 20, 256)  20992       input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 6, 6, 256)    5308672     conv1[0][0]                      
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 1152, 8)      0           conv2d_1[0][0]                   
______________________________________________

**We split the train data into training and validation sets using sklearn's *train_test_split* function. However it will be best to slit this in a way that best makes validation and train sets uncorrelated. I converted the dataset beforehand into hd5 format as drive takes a long time to directly access individual files.**

In [0]:
from sklearn.model_selection import train_test_split
import h5py

with h5py.File('drive/Datasets/traindata28.h5', 'r') as hf:
    X = hf['Images'][:]
    Y = hf['Labels'][:]
    
x_train, x_test, y_train, y_test = train_test_split(X,Y)
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
y_train = to_categorical(y_train.astype('float32'))
y_test = to_categorical(y_test.astype('float32'))

**Now we define the training function**

In [0]:
def train(model, data, epoch_size_frac=1.0,num=0):
    """
    Training a CapsuleNet
    :param model: the CapsuleNet model
    :param data: a tuple containing training and testing data, like `((x_train, y_train), (x_test, y_test))`
    :param args: arguments
    :return: The trained model
    """
    # unpacking the data
    (x_train, y_train), (x_test, y_test) = data

    # callbacks
    log = callbacks.CSVLogger('log.csv')
    checkpoint = callbacks.ModelCheckpoint('weights-{epoch:02d}.h5',
                                           save_best_only=True, save_weights_only=True, verbose=1)
    lr_decay = callbacks.LearningRateScheduler(schedule=lambda epoch: 0.001 * np.exp(-epoch / 10.))

    # compile the model
    model.compile(optimizer='adam',
                  loss=[margin_loss, 'mse'],
                  loss_weights=[1., 0.0005],
                  metrics={'out_caps': 'accuracy'})

    """
    # Training without data augmentation:
    model.fit([x_train, y_train], [y_train, x_train], batch_size=args.batch_size, epochs=args.epochs,
              validation_data=[[x_test, y_test], [y_test, x_test]], callbacks=[log, tb, checkpoint])
    """

    # -----------------------------------Begin: Training with data augmentation -----------------------------------#
    def train_generator(x, y, batch_size, shift_fraction=0.):
        train_datagen = ImageDataGenerator(width_shift_range=shift_fraction,
                                           height_shift_range=shift_fraction)  # shift up to 2 pixel for MNIST
        generator = train_datagen.flow(x, y, batch_size=batch_size)
        while 1:
            x_batch, y_batch = generator.next()
            yield ([x_batch, y_batch], [y_batch, x_batch])

    # Training with data augmentation. If shift_fraction=0., also no augmentation.
    model.fit_generator(generator=train_generator(x_train, y_train, 64, 0.1),
                        steps_per_epoch=int(epoch_size_frac*y_train.shape[0] / 64),
                        epochs=5,
                        validation_data=[[x_test, y_test], [y_test, x_test]])
    # -----------------------------------End: Training with data augmentation -----------------------------------#

    model.save_weights('drive/Datasets/model.h5')
    return model

**Since this is a cloud based service that is given "$free$" it is best to periodically save your models during training. Remember colab will not let you occupy a virtual machine for more than 6 hours and has the occasional tendency to disconnect in the middle of long processes **

In [13]:
for i in range(0,10):
    print(i)
    model.load_weights('drive/Datasets/model.h5')
    train(model=model, data=((x_train, y_train), (x_test, y_test)), num = i)

0
Epoch 1/5
 653/1120 [================>.............] - ETA: 2:02 - loss: 0.1011 - out_caps_loss: 0.1011 - out_recon_loss: 0.0377 - out_caps_acc: 0.8975

1120/1120 [==============================] - 308s 275ms/step - loss: 0.1010 - out_caps_loss: 0.1010 - out_recon_loss: 0.0376 - out_caps_acc: 0.8975 - val_loss: 0.0894 - val_out_caps_loss: 0.0894 - val_out_recon_loss: 0.0448 - val_out_caps_acc: 0.9156
Epoch 2/5
  84/1120 [=>............................] - ETA: 4:26 - loss: 0.1069 - out_caps_loss: 0.1068 - out_recon_loss: 0.0368 - out_caps_acc: 0.9022

 792/1120 [====================>.........] - ETA: 1:25 - loss: 0.0969 - out_caps_loss: 0.0968 - out_recon_loss: 0.0369 - out_caps_acc: 0.9020

1120/1120 [==============================] - 307s 274ms/step - loss: 0.0963 - out_caps_loss: 0.0963 - out_recon_loss: 0.0370 - out_caps_acc: 0.9030 - val_loss: 0.0857 - val_out_caps_loss: 0.0857 - val_out_recon_loss: 0.0445 - val_out_caps_acc: 0.9124
Epoch 3/5
 133/1120 [==>...........................] - ETA: 4:14 - loss: 0.1107 - out_caps_loss: 0.1107 - out_recon_loss: 0.0371 - out_caps_acc: 0.8832

 841/1120 [=====================>........] - ETA: 1:12 - loss: 0.0928 - out_caps_loss: 0.0928 - out_recon_loss: 0.0365 - out_caps_acc: 0.9042

...

1120/1120 [==============================] - 308s 275ms/step - loss: 0.0480 - out_caps_loss: 0.0480 - out_recon_loss: 0.0301 - out_caps_acc: 0.9535 - val_loss: 0.0470 - val_out_caps_loss: 0.0470 - val_out_recon_loss: 0.0387 - val_out_caps_acc: 0.9516
Epoch 4/5
 159/1120 [===>..........................] - ETA: 4:09 - loss: 0.0470 - out_caps_loss: 0.0469 - out_recon_loss: 0.0302 - out_caps_acc: 0.9545

 867/1120 [======================>.......] - ETA: 1:05 - loss: 0.0469 - out_caps_loss: 0.0469 - out_recon_loss: 0.0301 - out_caps_acc: 0.9556

1120/1120 [==============================] - 307s 274ms/step - loss: 0.0474 - out_caps_loss: 0.0474 - out_recon_loss: 0.0301 - out_caps_acc: 0.9551 - val_loss: 0.0462 - val_out_caps_loss: 0.0462 - val_out_recon_loss: 0.0390 - val_out_caps_acc: 0.9565
Epoch 5/5
 159/1120 [===>..........................] - ETA: 4:07 - loss: 0.0467 - out_caps_loss: 0.0466 - out_recon_loss: 0.0304 - out_caps_acc: 0.9553

 867/1120 [======================>.......] - ETA: 1:05 - loss: 0.0477 - out_caps_loss: 0.0477 - out_recon_loss: 0.0299 - out_caps_acc: 0.9547

1120/1120 [==============================] - 307s 274ms/step - loss: 0.0475 - out_caps_loss: 0.0475 - out_recon_loss: 0.0300 - out_caps_acc: 0.9549 - val_loss: 0.0518 - val_out_caps_loss: 0.0518 - val_out_recon_loss: 0.0391 - val_out_caps_acc: 0.9500


**Finally we get the test resuls.**

In [0]:
with h5py.File('drive/Datasets/testdata28.h5', 'r') as hf:
    data_test = hf['Images'][:]
    img_id = hf['Id'][:]

y_pred, _ = model.predict([data_test, 
                           np.zeros((data_test.shape[0],10))], # empty values for the second vector 
                           batch_size = 32, verbose = True)


In [0]:
y_pred = np.argmax(y_pred,1)
with open('submission.csv', 'w') as out_file:
    out_file.write('ImageId,Label\n')
    for i in range(len(y_pred)):
        out_file.write('%d,%d\n' % (img_id[i], y_pred[i]))

In [84]:
#@title Author

%%html
<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
<style>
img {
  float: left;
  align: left;
  border-radius: 50%;
}
div.container {
    width: 60%;
    align-content: left;
}
div.name {
    font-style: bold;
    text-align: center;
    font-size: 300%;
}
div.hobbies {
    text-align: center;
    color: #FF9900;
    size: 60%;
}
div.edu {
    padding-top: 10px;
    text-align: center;
    font-style: italic;
    color : Gray;
}
div.contact {
    padding-top: 15px;
    text-align: center;
    size: 80%;
}
div.quote {
    font-style: italic;
    text-align: center;
    font-size: 120%;
    color: Gray;
    padding-top: 30px;
}
    
    
</style>
</head>
<meta name="viewport" content="width=device-width, initial-scale=1">
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">
<body>
    
    <img src=https://drive.google.com/uc?id=1IWQEbXsOl6FpaPssexnxqA_xNDC3HRu1 style="width:200px;height:200px;">
    <div class="container">
        <div class="name">Samiul Alam</div>
        <div class = "hobbies">AI Enthusiast </div>
        <div class = "edu">Graduate, Bangladesh University of Engineering And Technology </div>
        <div class = "contact"><i class="fa fa-github"></i> samiul272 | <i class="fa fa-linkedin-square"> samiul-alam  | </i> <i class="fa fa-skype"> samiul272 </i> </div>
        <div class="quote">"Perpetually Convalescent: Learning and Evolving" </div> 
    </div>
</body>
</html>


